In this notebook we will have the code used to scrape the initial player data. <br>
At first I will be only scrapping the basic stats of the actual players in LEC in summer 2022 <br> <br>
**This notebook might be temporal, probably in the future I will change it with another notebook where I scrape more general data or all players**

In [1]:
# Importing libraries

import numpy as np
import pandas as pd
from datetime import datetime, date
import re

from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
df = pd.read_csv("C:/Users/adars/OneDrive/Escritorio/ProjecteLolShiny/Data/PlayerStatesTrajectory.csv")

In [7]:
df.iloc[:, :-2]

,Player,Year,Team,Position,Games Played,Win percentage,KDA,KP,DTH%,GD@10,XPD@10,CSPM,DPM,DMG%,GOLD%,WPM,WCPM
0,113,2023,Astralis,Jungle,42,40%,2.5,74.5%,22.2%,-183,-179,4.5,303,14.8%,16.9%,0.31,0.30
1,113,2022,Karmine Corp,Jungle,85,62%,3.0,67.9%,28.7%,86,-18,5.2,326,14.1%,15.9%,0.36,0.31
2,113,2021,Giants,Jungle,41,54%,2.6,71.6%,26.3%,10,-90,5.2,398,19.3%,18.2%,0.43,0.28
3,113,2021,SuperMassive Academy,Jungle,23,57%,4.5,73.7%,19.7%,212,88,6.0,369,21.2%,19.8%,0.35,0.32
4,113,2020,SuperMassive Academy,Jungle,32,44%,3.0,69.8%,23.1%,89,5,4.8,300,16.7%,18.0%,0.50,0.33
5,Elyoya,2023,MAD Lions,Jungle,74,51%,4.0,72.0%,17.2%,40,44,5.3,321,13.8%,18.3%,0.41,0.40
6,Elyoya,2022,MAD Lions,Jungle,57,51%,4.0,74.1%,18.9%,193,154,5.6,300,15.4%,18.7%,0.37,0.50
7,Elyoya,2021,MAD Lions,Jungle,91,62%,4.7,71.3%,17.0%,59,72,5.9,321,15.4%,18.7%,0.44,0.38
8,Elyoya,2021,MAD Lions,Middle,1,0%,1.8,64.7%,20.7%,-217,-483,6.7,503,26.3%,21.9%,0.18,0.18
9,Elyoya,2020,Movistar Riders,Jungle,72,72%,6.3,72.9%,15.8%,206,274,6.1,336,16.2%,19.2%,0.46,0.47


In [118]:
# Getting players to scrap (read csv file with LEC 2023 sumemmer season player stats)

playerStats_summerLEC2023 = pd.read_csv("./Data/LEC 2023 Summer Season - Player Stats.csv")
playerStats_summerLEC2023.head()

# Get the player names
players_name = playerStats_summerLEC2023["Player"].tolist()

##### Get basic data

In [119]:
# Scrap players data (all history)

players_name_test = ['113', 'Elyoya']

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
data_table = []

for player_name in players_name_test:
    url = f"https://oracleselixir.com/player/{player_name}/statsByYear"
    driver.get(url)

    # Find the table on the page (adjust the table selection based on the specific website structure)
    div_element = driver.find_elements(By.CSS_SELECTOR, "div.sc-dAEZTx.elXwcY")


    for item in div_element:
        data_list = item.text.split("\n")
        data_table.append([player_name] + data_list)

# Close the browser
driver.quit()

In [121]:
columns_name = ["Player", "Year", "Team", "Position", "Games Played", "Win percentage", "KDA", "KP", "DTH%", "GD@10", "XPD@10", "CSPM", "DPM", "DMG%", "GOLD%", "WPM", "WCPM"]
df_players = pd.DataFrame(data_table, columns = columns_name)

##### Get trajectory data

In [4]:
# Scrap players trajectory data

players_name_test = ['113', 'Elyoya', "Caps", "Razork", "Yike", "Exakick", "Upset", "Humanoid", "Hylissang", "Nisqy"]

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
trajectory_data = []

for i, player_name in enumerate(players_name_test):
    url = f"https://lol.fandom.com/wiki/{player_name}"
    driver.get(url)

    if i == 0:
        driver.implicitly_wait(5)
        driver.find_element(By.CSS_SELECTOR, 'div._2O--J403t2VqCuF8XJAZLK').click()

    table_element = driver.find_elements(By.CSS_SELECTOR, "table.player-team-history.hoverable-rows.sortable.jquery-tablesorter")

    aux_dict = {}

    for item in table_element:
        aux_dict["Player"] = player_name
        aux_teamname = item.find_elements(By.CSS_SELECTOR, "span.teamname")
        aux_dict['Teams trajectory'] = [i.text for i in aux_teamname]
        aux_dates = item.find_elements(By.CSS_SELECTOR, "span.ofl-toggle-2-1.ofl-toggler-2-all")
        aux_dict['Dates'] = [(aux_dates[i].text, aux_dates[i+1].text) for i in range(0, len(aux_dates) - 1, 2)]
        if len(aux_dates) % 2 != 0:
            aux_dict['Dates'].append((aux_dates[-1].text, "present"))


    trajectory_data.append(aux_dict)

# We pass it to dataframe
df_trajectory = pd.DataFrame(trajectory_data)

# Close the browser
driver.quit()

### Date format modification

In [5]:
def convert_date_range(date_str):

    start_date, end_date = date_str

    # Remove '≈' symbol from start and end dates
    start_date = start_date.replace('≈', '')
    end_date = end_date.replace('≈', '')

    # Convert start date
    if start_date == 'present':
        start_date = date.today().strftime('%Y-%m-%d')
    else:
        start_date = datetime.strptime(start_date, '%b %Y').date().strftime('%Y-%m-%d')

    # Convert end date
    if end_date == 'present':
        end_date = date.today().strftime('%Y-%m-%d')
    else:
        end_date = datetime.strptime(end_date, '%b %Y').date().strftime('%Y-%m-%d')

    return start_date, end_date

In [6]:
df_trajectory_exploded = df_trajectory.explode(["Teams trajectory", "Dates"])

In [7]:
df_trajectory_exploded['start_date'], df_trajectory_exploded["end_date"] = zip(*df_trajectory_exploded['Dates'].apply(convert_date_range))

In [8]:
df_trajectory_exploded

,Player,Teams trajectory,Dates,start_date,end_date
0,113,SUP Academy,"(Jan 2020, May 2021)",2020-01-01,2021-05-01
0,113,Vodafone Giants,"(May 2021, Nov 2021)",2021-05-01,2021-11-01
0,113,Karmine Corp,"(Nov 2021, Nov 2022)",2021-11-01,2022-11-01
0,113,Astralis,"(Dec 2022, present)",2022-12-01,2023-07-21
1,Elyoya,iNAT Gaming,"(Dec 2018, ≈Mar 2019)",2018-12-01,2019-03-01
...,...,...,...,...,...
9,Nisqy,Splyce,"(Nov 2017, Nov 2018)",2017-11-01,2018-11-01
9,Nisqy,Cloud9,"(Nov 2018, Nov 2020)",2018-11-01,2020-11-01
9,Nisqy,Fnatic,"(Nov 2020, Dec 2021)",2020-11-01,2021-12-01
9,Nisqy,Fnatic,"(Dec 2021, May 2022)",2021-12-01,2022-05-01


In [9]:
df_trajectory_exploded.to_csv('C:/Users/adars/OneDrive/Escritorio/ProjecteLolShiny/Data/PlayersTrajectoyData.csv', index=False)